In [237]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [238]:
df2 = pd.read_csv('./model/tmdb.csv')

In [239]:
df2.head(2)

,Unnamed: 0,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,status,tagline,title,vote_average,vote_count,tittle,cast,crew,director,soup
0,0,237000000,"['action', 'adventure', 'fantasy']",http://www.avatarmovie.com/,19995,"['cultureclash', 'future', 'spacewar']",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"['samworthington', 'zoesaldana', 'sigourneywea...","[{'credit_id': '52fe48009251416c750aca23', 'de...",jamescameron,cultureclash future spacewar samworthington zo...
1,1,300000000,"['adventure', 'fantasy', 'action']",http://disney.go.com/disneypictures/pirates/,285,"['ocean', 'drugabuse', 'exoticisland']",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"['johnnydepp', 'orlandobloom', 'keiraknightley']","[{'credit_id': '52fe4232c3a36847f800b579', 'de...",goreverbinski,ocean drugabuse exoticisland johnnydepp orland...


In [240]:
df2.shape

(4803, 26)

In [241]:
df2 = df2.drop('soup',axis=1)
df2.head(2)

,Unnamed: 0,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew,director
0,0,237000000,"['action', 'adventure', 'fantasy']",http://www.avatarmovie.com/,19995,"['cultureclash', 'future', 'spacewar']",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"['samworthington', 'zoesaldana', 'sigourneywea...","[{'credit_id': '52fe48009251416c750aca23', 'de...",jamescameron
1,1,300000000,"['adventure', 'fantasy', 'action']",http://disney.go.com/disneypictures/pirates/,285,"['ocean', 'drugabuse', 'exoticisland']",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"['johnnydepp', 'orlandobloom', 'keiraknightley']","[{'credit_id': '52fe4232c3a36847f800b579', 'de...",goreverbinski


In [242]:
features = ['keywords', 'cast', 'genres', 'director', 'tagline']

In [243]:
for feature in features:
    df2[feature] = df2[feature].fillna('')

In [244]:
def combined_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']+row['tagline']
df2["combined_features"] = df2.apply(combined_features, axis =1)

In [245]:
df2.head(2)

,Unnamed: 0,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,status,tagline,title,vote_average,vote_count,tittle,cast,crew,director,combined_features
0,0,237000000,"['action', 'adventure', 'fantasy']",http://www.avatarmovie.com/,19995,"['cultureclash', 'future', 'spacewar']",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"['samworthington', 'zoesaldana', 'sigourneywea...","[{'credit_id': '52fe48009251416c750aca23', 'de...",jamescameron,"['cultureclash', 'future', 'spacewar'] ['samwo..."
1,1,300000000,"['adventure', 'fantasy', 'action']",http://disney.go.com/disneypictures/pirates/,285,"['ocean', 'drugabuse', 'exoticisland']",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"['johnnydepp', 'orlandobloom', 'keiraknightley']","[{'credit_id': '52fe4232c3a36847f800b579', 'de...",goreverbinski,"['ocean', 'drugabuse', 'exoticisland'] ['johnn..."


In [246]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['combined_features'])

In [247]:
def get_recommendations(title):
    cosine_sim = cosine_similarity(count_matrix)
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    sim_scores1 = [i[1] for i in sim_scores]    
    tit = df2['title'].iloc[movie_indices]
    dat = df2['release_date'].iloc[movie_indices]
    return_df = pd.DataFrame(columns=['Title','Year','sim_scores'])
    return_df['Title'] = tit
    return_df['Year'] = dat
    return_df['sim_scores'] = sim_scores1
    return return_df

In [248]:
get_recommendations('Avatar')

,Title,Year,sim_scores
1,Pirates of the Caribbean: At World's End,2007-05-19,0.360844
50,Prince of Persia: The Sands of Time,2010-05-19,0.348155
206,Clash of the Titans,2010-04-01,0.348155
322,The Fifth Element,1997-05-07,0.348155
131,G-Force,2009-07-21,0.320256
46,X-Men: Days of Future Past,2014-05-15,0.308607
466,The Time Machine,2002-03-04,0.308607
1438,Krull,1983-07-29,0.308607
786,The Monkey King 2,2016-01-22,0.306186
836,The Forbidden Kingdom,2008-04-18,0.288675


In [249]:
get_recommendations('Show Me')

,Title,Year,sim_scores
3431,Layer Cake,2004-09-30,0.358569
1051,Prisoners,2013-09-18,0.327327
3556,10 Cloverfield Lane,2016-03-10,0.327327
538,Hostage,2005-03-09,0.314485
2915,Trash,2014-10-09,0.314485
2385,I Come with the Rain,2009-05-14,0.308607
2764,Light It Up,1999-11-04,0.308607
724,Man on Fire,2004-04-23,0.303046
1280,Disturbia,2007-04-12,0.303046
3507,Deadfall,2012-11-08,0.292770


In [250]:
get_recommendations('Hackers')

,Title,Year,sim_scores
468,Swordfish,2001-06-07,0.402911
512,Wanted,2008-06-19,0.322329
1133,15 Minutes,2001-03-01,0.308607
4509,Love Letters,1983-04-01,0.308607
3714,Exiled,2006-09-06,0.303046
4256,Checkmate,2015-07-03,0.303046
273,Gone in Sixty Seconds,2000-06-09,0.296500
2200,Crank: High Voltage,2009-04-16,0.296500
613,The Score,2001-07-13,0.285714
1177,Sin City,2005-04-01,0.285714
